# For each of the best seller book mentioned on the first two pages of the amazon bestsellers, scrape the following information:
i. Publisher
ii. Language
iii. Paperback
iv. ISBN-10
v. ISBN-13
vi. Weight
vii. Dimensions
viii. First Three Customer Reviews

In [1]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np

In [2]:
def get_links(page_no):
    url=('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(page_no)+'?ie=UTF8&pg='+str(page_no))
    req=requests.get(url)
    html=req.content
    page_soup=soup(html) 
    allLinks=[]
    
    for book in page_soup.find_all('div',attrs={'class':'a-section a-spacing-none aok-relative'}):
        book_link=book.find('a',class_='a-link-normal')
        links=[]   
        if book_link is not None:
            links.append('https://www.amazon.in'+book_link.get('href'))
        else:
            links.append(np.nan)
        allLinks.append(links)

    return allLinks

In [3]:
links=[]
for i in range(1,3):
    links.append(get_links(i))
flatten = lambda l: [item for sublist in l for item in sublist]
links_df=pd.DataFrame(flatten(links),columns=['links'])

In [5]:
def get_reviews(soup):
    reviews=[]
    str=""
    for r in soup.find_all("div",class_="a-expander-content reviewText review-text-content a-expander-partial-collapse-content")[0:3]:
        str=str+r.get_text()        
    res=str.split("\n")
    for i in res:
        if i == "":
            pass
        else:
            reviews.append(i.strip())
    return reviews[0:3]

In [6]:
def get_product_details(books_soup):
    names,details,books_details=[],[],[]

    product_details=books_soup.find(id="detailBullets_feature_div")
    for name in product_details.find_all('span',class_="a-text-bold"):
        names.append(name.getText().encode('ascii', 'ignore').decode('unicode_escape').split('\n')[0])    
    for detail in product_details.find_all('span',class_=""):
        details.append(detail.getText().strip())
    
    details_dict = dict(zip(names, details))
    books_details.append(details_dict.get('Publisher'))
    books_details.append(details_dict.get('Language'))
    if(details_dict.get('Paperback')!=None):
        books_details.append('Paperback: '+details_dict.get('Paperback'))
    elif(details_dict.get('Hardcover')!=None):
        books_details.append('Hardcover: '+details_dict.get('Hardcover'))
    else: books_details.append(None)
    books_details.append(details_dict.get('ISBN-10'))
    books_details.append(details_dict.get('ISBN-13'))
    books_details.append(details_dict.get('Item Weight'))
    books_details.append(details_dict.get('Dimensions'))
   
    return books_details   

In [7]:
def get_books_details(books_soup):
    books_details,reviews,books=[],[],[]
    books_details=get_product_details(books_soup)
    reviews=get_reviews(books_soup)
    books.append(books_details+reviews)

    return books 

In [8]:
all_books_details=[]
for i in range(len(links_df)):
    url=links_df.loc[i,'links']
    
    HEADERS = ({'User-Agent': 'Chrome/90.0.4430.212 Safari/537.36','Accept-Language': 'en-US'})
    response=requests.get(url,headers=HEADERS)
    print(response.status_code)
    books_soup=soup(response.content)
    all_books_details.append(get_books_details(books_soup))

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [9]:
flatten= lambda l:[item for sublist in l for item in sublist] 
books_df=pd.DataFrame(flatten(all_books_details))
books_df.columns=['Publisher','Language','Paperback','ISBN-10','ISBN-13','Weight','Dimensions','Review-1','Review-2','Review-3']

In [10]:
books_df

,Publisher,Language,Paperback,ISBN-10,ISBN-13,Weight,Dimensions,Review-1,Review-2,Review-3
0,Jaico Publishing House; First edition (5 July ...,English,Paperback: 352 pages,9788184955699,978-8184955699,400 g,15.2 x 2.01 x 22.9 cm,Nice book but some page was missing. 185-216 m...,I was assuming to receive a big sized book.But...,Elaborate and well Oriented topices.It will he...
1,Fingerprint! Publishing; Classic edition (1 Ja...,English,Paperback: 328 pages,8172345194,978-8172345198,240 g,20 x 14 x 4 cm,Wonderful book having described different aspe...,Social/Historical context:The diary was writte...,Anne's letters are without a doubt worthy of r...
2,MTG Learning Media (P) Ltd. (21 July 2020); MT...,English,Paperback: 1264 pages,9389971241,978-9389971248,1 kg 830 g,22 x 17.5 x 1.5 cm,In very good condition with proper sequence of...,The product is so awesome this book is best fo...,"Must have for Neet aspirant❤ Snapshots, Colour..."
3,Fingerprint! Publishing (1 February 2015),English,Paperback: 256 pages,8172345666,978-8172345662,280 g,20 x 14 x 4 cm,Let me tell you about what the book says: Your...,SPOILERS AHEAD PEOPLE!!Murphy's idea is very s...,Very useful book...good for maintaining a calm...
4,"Diamond Books X-30, Okhla Industrial Area, Pha...",English,Paperback: 128 pages,9354865518,978-9354865510,200 g,21.59 x 13.97 x 0.75 cm,360 Pegree Postural Medicine is a book which c...,None,None
5,RHUK; Exported edition (7 February 2008),English,Paperback: 160 pages,9781846041242,978-1846041242,130 g,11 x 1 x 17.8 cm,THIS BOOK REVIEW IS ONLY BASED ON MY FEELINGS ...,"The book says, ‘The Classic tribute to hope fr...",I came across this book from a brief mentionin...
6,Manjul Publishing House; 2017th edition (1 Sep...,Hindi,Paperback: 320 pages,8186775218,978-8186775219,290 g,25 x 25 x 3 cm,This book is very useful for the younger peopl...,"I heard a lot about this amazing book, someday...",Very very fast delivery.I will start reading f...
7,Michael O'Mara Books; Illustrated edition (12 ...,English,Paperback: 192 pages,1780552491,978-1780552491,150 g,12.7 x 1.27 x 20.32 cm,"Purchased : Jan 2018, Current Usage: A week to...","Bought it for my 8 year old son, he can do onl...",Interesting and challenging book. Good for 8 ...
8,S Chand Publishing; Regular edition (1 January...,English,Paperback: 384 pages,9352530144,978-9352530144,662 g,20.3 x 25.4 x 4.7 cm,👍,Exercises without answer keys is useless. The ...,Quality of Paper-5Delivery-5Very useful for st...
9,Fingerprint! Publishing; First edition (1 Janu...,English,Paperback: 448 pages,8172343116,978-8172343118,300 g,20 x 14 x 4 cm,"First, I must admit a fact, infact, a couple.1...",a must for the youth who swear by distorted fa...,MK Gandhi has written a beautiful memoir of hi...


In [15]:
# books_df.to_csv('amazon_bestsellers_details.csv')

#  Scrape the IMDB information about movies released in the year 2020 (sorted according to number of votes) for the first 1000 movies.

In [2]:
from bs4 import BeautifulSoup as soup
import requests
import numpy as np
import pandas as pd

In [3]:
def get_movies_details(pn):
    url="https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start="+str(pn)
    req=requests.get(url)
    page=soup(req.content)
    movies_info=[]

    for movieFrame in page.find_all('div',attrs={"class":"lister-item mode-advanced"}):
        movies=[]
        directors=[]
        stars=[]

        movie_name=movieFrame.find("h3",class_="lister-item-header").find("a")
        imdb_rating=movieFrame.find('div',attrs={"class":"inline-block ratings-imdb-rating"})
        metascore=movieFrame.find("span",class_="metascore favorable")
        desc=movieFrame.find_all("p", class_="text-muted")[-1]
        
        vote_col=movieFrame.find("p",class_="sort-num_votes-visible")
        votes=vote_col.find_all("span",attrs={"name":"nv"})[0]

        movieCast=movieFrame.find("p",class_="")
        genre=movieFrame.find("span",class_="genre")
        runtime=movieFrame.find("span",class_="runtime")
       
        if movie_name is not None:
            movies.append(movie_name.text.lstrip())
        else:
            movies.append(np.nan)        
        if imdb_rating is not None:
            movies.append(imdb_rating.text.strip())
        else:
            movies.append(np.nan)
        if metascore is not None:
            movies.append(metascore.text.rstrip())
        else:
            movies.append(np.nan)
        if desc is not None:
            movies.append(desc.text.strip())
        else:
            movies.append(np.nan)
        if votes is not None:
            movies.append(votes.text)
        else:
            movies.append(np.nan)
        
        if movieCast is not None:
            directors,stars=get_cast(movieCast)
            movies.append("".join(map(str,directors)))
            movies.append("".join(map(str,stars)))
        else: movies.append(np.nan)

        if genre is not None:
            movies.append(genre.text.strip())
        else:
            movies.append(np.nan)
        if runtime is not None:
            movies.append(runtime.text)
        else:
            movies.append(np.nan)
        movies_info.append(movies)
        # break
    return movies_info


In [4]:
def get_cast(movieCast):
    all_cast=[]
    all_directors=[]
    all_stars=[]
    try:
        casts=movieCast.text.replace("\n","").split('|')
        casts = [x.strip() for x in casts]
        # directors
        directors="".join(map(str,casts[0]))
        first_ele=directors.partition(":")
        if(first_ele[0]=="Stars"):
            direct=np.nan
        else:
            direct=first_ele[2]
        all_directors.append(direct)
        # stars
        star_cast="".join(map(str,casts))
        dirs = str(star_cast).partition(":")
        if(dirs[0]!="Stars"):
            stars=str(dirs[2]).partition(":")[2]
        else: stars=dirs[2]
        all_stars.append(stars)
        
        all_cast.append(all_directors)
        all_cast.append(all_stars)
    except:
        casts = movieCast.text.replace("\n","").strip()
        print(np.nan)
    return all_directors,all_stars
    

In [5]:
movies_data=[]
for i in [1,51]:
    movies_data.append(get_movies_details(i))
flatten= lambda l:[item for sublist in l for item in sublist] 

In [6]:
cols=['Movie name','IMDB Rating','Metascore','Description','Votes','Director(s)','Stars','Movie Genre','Total Duration']
movies_df=pd.DataFrame(flatten(movies_data),columns=cols)
movies_df=movies_df.style.set_properties(**{'text-align': 'left'})
movies_df=movies_df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

In [7]:
movies_df

,Movie name,IMDB Rating,Metascore,Description,Votes,Director(s),Stars,Movie Genre,Total Duration
0,Tenet,7.4,69,"Armed with only one word, Tenet, and fighting for the survival of the entire world, a Protagonist journeys through a twilight world of international espionage on a mission that will unfold in something beyond real time.","435,800",Christopher Nolan,"John David Washington, Robert Pattinson, Elizabeth Debicki, Juhan Ulfsak","Action, Sci-Fi, Thriller",150 min
1,The Queen's Gambit,8.6,nan,"Orphaned at the tender age of nine, prodigious introvert Beth Harmon discovers and masters the game of chess in 1960s USA. But child stardom comes at a price.","364,945",nan,"Anya Taylor-Joy, Chloe Pirrie, Bill Camp, Marcin Dorocinski",Drama,395 min
2,Soul,8.1,83,"After landing the gig of a lifetime, a New York jazz pianist suddenly finds himself trapped in a strange land between Earth and the afterlife.","285,008","Pete Docter, Kemp Powers","Jamie Foxx, Tina Fey, Graham Norton, Rachel House","Animation, Adventure, Comedy",100 min
3,Wonder Woman 1984,5.4,nan,"Diana must contend with a work colleague and businessman, whose desire for extreme wealth sends the world down a path of destruction, after an ancient artifact that grants wishes goes missing.","237,870",Patty Jenkins,"Gal Gadot, Chris Pine, Kristen Wiig, Pedro Pascal","Action, Adventure, Fantasy",151 min
4,Birds of Prey: And the Fantabulous Emancipation of One Harley Quinn,6.1,nan,"After splitting with the Joker, Harley Quinn joins superheroines Black Canary, Huntress and Renee Montoya to save a young girl from an evil crime lord.","214,206",Cathy Yan,"Margot Robbie, Rosie Perez, Mary Elizabeth Winstead, Jurnee Smollett","Action, Adventure, Comedy",109 min
5,The Invisible Man,7.1,72,"When Cecilia's abusive ex takes his own life and leaves her his fortune, she suspects his death was a hoax. As a series of coincidences turn lethal, Cecilia works to prove that she is being hunted by someone nobody can see.","198,485",Leigh Whannell,"Elisabeth Moss, Oliver Jackson-Cohen, Harriet Dyer, Aldis Hodge","Drama, Horror, Mystery",124 min
6,Extraction,6.7,nan,"Tyler Rake, a fearless black market mercenary, embarks on the most deadly extraction of his career when he's enlisted to rescue the kidnapped son of an imprisoned international crime lord.","182,624",Sam Hargrave,"Chris Hemsworth, Bryon Lerum, Ryder Lerum, Rudhraksh Jaiswal","Action, Thriller",116 min
7,A Quiet Place Part II,7.3,71,"Following the events at home, the Abbott family now face the terrors of the outside world. Forced to venture into the unknown, they realize the creatures that hunt by sound are not the only threats lurking beyond the sand path.","174,592",John Krasinski,"Emily Blunt, Millicent Simmonds, Cillian Murphy, John Krasinski","Drama, Horror, Sci-Fi",97 min
8,The Trial of the Chicago 7,7.8,76,"The story of 7 people on trial stemming from various charges surrounding the uprising at the 1968 Democratic National Convention in Chicago, Illinois.","161,238",Aaron Sorkin,"Eddie Redmayne, Alex Sharp, Sacha Baron Cohen, Jeremy Strong","Drama, History, Thriller",129 min
9,Enola Holmes,6.6,68,"When Enola Holmes-Sherlock's teen sister-discovers her mother missing, she sets off to find her, becoming a super-sleuth in her own right as she outwits her famous brother and unravels a dangerous conspiracy around a mysterious young Lord.","151,923",Harry Bradbeer,"Millie Bobby Brown, Henry Cavill, Sam Claflin, Helena Bonham Carter","Action, Adventure, Crime",123 min


In [ ]:
# movies_df.to_csv('IMDB_movies.csv')